<a href="https://colab.research.google.com/github/tomonari-masada/course2021-stats1/blob/main/minibatch_PLSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLSIのミニバッチ学習

## MeCabのインストール

In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
#!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [ ]:
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

## データの取得

* ライブドア・ニュース・コーパスの前処理については下の記事を参考にした。
 * https://tech.fusic.co.jp/posts/2021-04-23-bert-multi-classification/

In [ ]:
import os
import urllib.request
import re
import csv
import tarfile
import torch
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# データのダウンロード（カレントディレクトリに圧縮ファイルがダウンロードされる）
urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")

# ダウンロードした圧縮ファイルのパスを設定
tgz_fname = "ldcc-20140209.tar.gz" 

#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

cuda:0


In [ ]:
fname_class_list = {
  "dokujo-tsushin": [],
  "it-life-hack": [],
  "kaden-channel": [],
  "livedoor-homme": [],
  "movie-enter": [],
  "peachy": [],
  "smax": [],
  "sports-watch": [],
  "topic-news": []
}
target_genres = list(fname_class_list.keys())

In [ ]:
def remove_brackets(inp):
  # 記号とかを除く
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  output = re.sub(brackets_head, '', re.sub(brackets_tail, '', inp))
  return output

def read_body(f):
  # 2行スキップ
  next(f) # URL
  next(f) # タイムスタンプ
  next(f) # タイトル
  lines = [line.decode('utf-8').strip() for line in f]
  body = ' '.join(lines)
  body = remove_brackets(body)
  return body

# all_text.tsvを作る
with tarfile.open(tgz_fname) as tf:
  # 対象ファイルの選定
  for ti in tf:
    """
    ・ライセンスファイルはスキップ
    ・genre内のtxt意外ならスキップ
    ・txtファイル意外ならスキップ
    ・用意したgenre意外ならスキップ
    """
    if "LICENSE.txt" in ti.name:
      continue
    if len(ti.name.split('/')) < 3:
      continue
    if not ti.name.endswith(".txt"):
      continue
    genre = ti.name.split('/')[1]
    if not genre in target_genres:
      continue
        
    genre_index = target_genres.index(genre)
    fname_class_list[target_genres[genre_index]].append(ti.name)

  with open(tsv_fname, "w") as wf:
    writer = csv.writer(wf, delimiter='\t')
    for i, genre in enumerate(target_genres):
      for fname in fname_class_list[genre]:
        f = tf.extractfile(fname)
        row = [genre, i, read_body(f)]
        writer.writerow(row)

In [ ]:
# 作成したデータの読み込み
df = pd.read_csv("all_text.tsv", delimiter='\t', header=None, names=['media_name', 'label', 'body'])
df = df.dropna(how='any') # nanのところは落とす

# データの確認
print(f'データサイズ： {df.shape}')
display(df.sample(10))

データサイズ： (7367, 3)


,media_name,label,body
7352,topic-news,8,24日に放送された「ひるおび！」（TBS系列）では、金正日の専属料理人を務め、その後帰国した...
7295,topic-news,8,ロンドン五輪に出場するサッカー・U-23日本代表は、21日にメキシコ代表との親善試合に臨み、...
5078,smax,6,展示会イベント恒例のおねいさん写真のコーナーでございます 国内最大級の携帯電話や無線技術に...
6395,sports-watch,7,現在、大阪・長居スタジアムで行なわれているサッカー国際親善試合＝日本代表×アイスランド代表戦...
5992,sports-watch,7,今月1日、日本学生野球憲章の改正後初となる、プロ野球チーム（巨人・二軍）と大学単独チームによ...
4583,peachy,5,年に一度のスイーツとファッションの祭典、「TOKYO SWEETS COLLECTION 2...
5942,sports-watch,7,4日放送、テレビ朝日「報道ステーション」では、ドイツ・ブンデスリーガのドルトムントに所属する...
3245,movie-enter,4,アジア最大の映画の祭典「第24回東京国際映画祭（TIFF）」の開催日程が迫る中、学生応援団の...
1499,it-life-hack,1,niconicoは、ニコニコ動画に投稿された動画の中からユーザー投票により優秀な作品を決定す...
7031,topic-news,8,19日に「日経トレンディNET」に掲載された記事「カシオの不人気デジカメが台湾や中国で大ブレ...


## トークン化
* MeCabを使ってトークン化する。

In [ ]:
import MeCab

m = MeCab.Tagger()

In [ ]:
m.parse(df.body[0])

'もうすぐ\t副詞,一般,*,*,*,*,もうすぐ,モウスグ,モースグ\nジューン\t名詞,固有名詞,人名,名,*,*,ジューン,ジューン,ジューン\n・\t記号,一般,*,*,*,*,・,・,・\nブライド\t名詞,一般,*,*,*,*,ブライド,ブライド,ブライド\nと\t助詞,格助詞,引用,*,*,*,と,ト,ト\n呼ば\t動詞,自立,*,*,五段・バ行,未然形,呼ぶ,ヨバ,ヨバ\nれる\t動詞,接尾,*,*,一段,基本形,れる,レル,レル\n６月\t名詞,副詞可能,*,*,*,*,６月,ロクガツ,ロクガツ\n。\t記号,句点,*,*,*,*,。,。,。\n独\t名詞,固有名詞,地域,国,*,*,独,ドク,ドク\n女\t名詞,一般,*,*,*,*,女,オンナ,オンナ\nの\t助詞,連体化,*,*,*,*,の,ノ,ノ\n中\t名詞,非自立,副詞可能,*,*,*,中,ナカ,ナカ\nに\t助詞,格助詞,一般,*,*,*,に,ニ,ニ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n自分\t名詞,一般,*,*,*,*,自分,ジブン,ジブン\nの\t助詞,連体化,*,*,*,*,の,ノ,ノ\n式\t名詞,一般,*,*,*,*,式,シキ,シキ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\nまだ\t副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ\nな\t助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ\nのに\t助詞,接続助詞,*,*,*,*,のに,ノニ,ノニ\n呼ば\t動詞,自立,*,*,五段・バ行,未然形,呼ぶ,ヨバ,ヨバ\nれ\t動詞,接尾,*,*,一段,連用形,れる,レ,レ\nて\t助詞,接続助詞,*,*,*,*,て,テ,テ\nばかり\t助詞,副助詞,*,*,*,*,ばかり,バカリ,バカリ\n…\t記号,一般,*,*,*,*,…,…,…\n…\t記号,一般,*,*,*,*,…,…,…\nという\t助詞,格助詞,連語,*,*,*,という,トイウ,トユウ\n「\t記号,括弧開,*,*,*,*,「,「,「\nお祝い\t名詞,サ変接続,*,*,*,*,お祝い,オイワイ,オイワイ\n貧乏\t名詞,サ変接続,*,*,*,*,貧乏,ビンボウ,ビンボー\n」\t記号,括弧閉,*,*,*,*,」,」,」\n状態\t名詞,一般,*,*,*,*,状態,ジョウタイ,ジョータイ

In [ ]:
def tokenize(text):
  tokens = []
  for line in m.parse(text).splitlines():
    fields = line.split()
    if len(fields) != 2: continue
    subfields = fields[1].split(',')
    if len(subfields) != 9: continue
    if subfields[0] in ['記号', '助詞', '助動詞', '連体詞', '副詞']: continue
    token = subfields[6]
    if token == '*':
      token = fields[0]
    tokens.append(token)
  return ' '.join(tokens)

In [ ]:
corpus = []
for body in df.body:
  corpus.append(tokenize(body))

* 文書数は7,367件

In [ ]:
len(corpus)

7367

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=20, max_df=0.2)
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names_out()

In [ ]:
len(vocab)

6682

## PLSIのミニバッチ学習
* PyTorchで実装する。

In [ ]:
import torch

In [ ]:
word_freqs = torch.from_numpy(X.toarray()).type(torch.float32)

In [ ]:
n_components = 20
theta = torch.randn(len(corpus), n_components, requires_grad=True)
phi = torch.randn(n_components, len(vocab), requires_grad=True)

In [ ]:
optimizer = torch.optim.Adam([theta, phi], lr=1.0)

In [ ]:
for epoch in range(100):
  normalized_theta = torch.nn.functional.softmax(theta, dim=1)
  normalized_phi = torch.nn.functional.softmax(phi, dim=1)
  word_probs = normalized_theta @ normalized_phi

  loss = - (word_freqs * word_probs.log()).sum()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  print(f'epoch {epoch} | loss {loss}')

epoch 0 | loss 10158444.0
epoch 1 | loss 9465356.0
epoch 2 | loss 9256895.0
epoch 3 | loss 9259055.0
epoch 4 | loss 9241736.0
epoch 5 | loss 9139079.0
epoch 6 | loss 9001437.0
epoch 7 | loss 8886162.0
epoch 8 | loss 8803339.0
epoch 9 | loss 8740564.0
epoch 10 | loss 8687714.0
epoch 11 | loss 8645815.0
epoch 12 | loss 8613696.0
epoch 13 | loss 8587228.0
epoch 14 | loss 8565194.0
epoch 15 | loss 8546052.0
epoch 16 | loss 8528226.0
epoch 17 | loss 8512706.0
epoch 18 | loss 8498872.0
epoch 19 | loss 8486700.0
epoch 20 | loss 8475234.0
epoch 21 | loss 8465093.0
epoch 22 | loss 8456704.0
epoch 23 | loss 8448814.0
epoch 24 | loss 8441327.0
epoch 25 | loss 8434526.0
epoch 26 | loss 8428213.0
epoch 27 | loss 8422241.0
epoch 28 | loss 8416494.0
epoch 29 | loss 8411151.0
epoch 30 | loss 8406438.0
epoch 31 | loss 8402220.0
epoch 32 | loss 8398347.0
epoch 33 | loss 8394302.0
epoch 34 | loss 8390449.0
epoch 35 | loss 8386993.0
epoch 36 | loss 8383840.5
epoch 37 | loss 8381025.0
epoch 38 | loss 83784

## トピック語の確認

In [ ]:
n_topic_words = 30
for k in range(n_components):
  print(' '.join([vocab[idx] for idx in torch.argsort(phi[k], descending=True)[:n_topic_words]]))

応募 当選 キャンペーン プレゼント くださる いただく 場合 賞品 発送 期間 連絡 了承 いたす 抽選 事項 購入 終了 予定 方法 注意 弊社 住所 セット 必要 内容 実施 試写 招待 参加 個人
更新 ソフトウェア 場合 くださる 利用 ダウンロード アップデート 設定 データ 表示 実施 電話 操作 端末 本体 必要 以下 手順 とき 電池 お客様 通信 ドコモ 注意 状態 方法 接続 いただく 画面 起動
東京 開催 写真 イベント 参加 自転車 デザイン 公式 楽しむ ファッション 開始 場所 配信 展示 行く 予定 女子 ツリー 会場 企画 体験 テーマ ホテル 走る 今年 楽しい 感じる 好き そして 実施
画面 表示 設定 機能 入力 ファイル 利用 便利 登録 クリック タップ 知る 起動 ページ データ 検索 選択 得る 場合 方法 文字 操作 簡単 無料 編集 ニュース チェック ユーザー インストール ダウンロード
写真 撮影 カメラ ビデオ 撮る 映像 動画 チェック 作品 ネット 編集 画像 売れ筋 レンズ 音楽 とき デジタル ソニー ムービー 節電 使用 映画 理由 作る ロゴ 簡単 電気 挑戦 知る 利用
仕事 転職 会社 やる 考える あなた 求人 経験 年収 聞く 営業 悩み 社員 働く でも 感じる 良い 行く 採用 必要 活動 面白い とき 違う 判断 すごい しれる 探す 分かる ビール
映画 公開 作品 監督 演じる 映像 主演 特集 描く 全国 観る アクション シーン そして 劇場 シリーズ 決定 ドラマ 女優 ファン 注目 公式 物語 アメリカ 主人公 出演 ストーリー 舞台 原作 記録
ネット 韓国 掲示板 批判 ニュース 発言 ユーザー コメント 寄せる 社長 番組 受ける 意見 問題 報じる 報道 放送 一方 ファン 内容 相次ぐ やる ちゃん よる 出る 事件 掲載 自身 写真 メディア
スマート フォン ドコモ モデル 搭載 対応 機能 機種 カメラ 予定 向け 本体 インチ ディスプレイ 開始 サイズ 画素 防水 開発 サービス 端末 リンク コア 最大 執筆 画面 動画 連続 タブレット 撮影
ドラマ 好き 彼女 映画 女子 クリスマス すごい 作品 観る 演じる 男性 行く 子供 シーン 描く

# 課題
* このバッチ最適化によるパラメータ推定を、ミニバッチ最適化に書き換えてみよう。
 * 同じ方法でlossを計算して比較することで、よりlossを小さくできるか、確認してみる。
* もしくは、授業で説明したEMアルゴリズムによる推定を実装し、今回のバッチ最適化と性能比較してみよう。